# Variational Quantum Eigensolver for the Quadratic Assignment Problem

In [2]:
#Initialisation cell
from docplex.mp.model import Model
from qiskit.providers.ibmq.managed import IBMQJobManager

from qiskit import BasicAer, Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver,NumPyEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import RealAmplitudes #(uses CX entangling) 
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.circuit.library import TwoLocal
#from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
from qiskit import IBMQ
from docplex.mp.model import Model
from qiskit.optimization import QuadraticProgram
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.applications.ising import docplex

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import pandas as pd

The functions below are for inputting the csv data (that is in the QAPLIB format) and making the matrices numpy arrays.

In [3]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [4]:
#small sized matrices(under 10x10) (quick on all methods)
matrix_size_3 = './data/made3.csv'
matrix_size_4 = './data/made4.csv'
matrix_size_5 = './data/made5.csv'
matrix_size_6 = './data/made6.csv'
matrix_size_7 = './data/made7.csv'
matrix_size_8 = './data/made8.csv'
matrix_size_9 = './data/made9.csv'

#medium sized matrices(ranging from 10x10 to 30x30) (slow on deterministic methods, fast on heuristics)
matrix_size_10 = './data/tai10a.csv'
matrix_size_11 = './data/made11.csv'
matrix_size_12 = './data/tai12a.csv'
matrix_size_15 = './data/chr15a.csv' 
matrix_size_20 = './data/chr20a.csv'
matrix_size_26 = './data/bur26a.csv'

#large sized matrices(30x30 and bigger)(reasonably slow on the heuristics to a certain degree of accuracy)
matrix_size_40 = './data/tai40a.csv'
matrix_size_60 = './data/tai60.csv'
matrix_size_80 = './data/tai80.csv'
matrix_size_256 = './data/tai256c.csv'

datamatrix = CSVtoNumpyArray(matrix_size_4) # Decide the size of problem to run in the code (clue: 
                                                #the number in the original name is the size)
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]

Below is the fitness function for the QAP using the  Quadratic 0–1 formulation

In [5]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """Compute the TSP value of a solution.
    Args:
        z (list[int]): list of allocations
        MatrixLoc (numpy array): matrix of distances
        MatrixFlow (numpy array): matrix of flow
    Returns:
        float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length))
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

Below, IBMQ account is loaded and the backends displayed

In [8]:


 # Load account from disk

IBMQ.save_account("3cd4df76e664d15c0a0dde90b7ce30dd1c5f0040004c5b46c8f6055623a104b2cc23d8473c156efa1dfc6c12fbb052c1c5b25a7b9f2ad26585ebb1a93f2be810", overwrite = True)
IBMQ.load_account()
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='internal', project='physics')>]

In [9]:
provider = IBMQ.get_provider(hub='ibm-q-wits', group='internal', project='physics')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_johannesburg') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_rochester') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('

These functions are needed to find and convert the answer

In [6]:
def qap_feasible(x):
    """Check whether a solution is feasible or not.

    Args:
        x (numpy.ndarray) : binary string as numpy array.

    Returns:
        bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n))
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [7]:
def choose_best_feasible(eigenstates):
    """
    Input:
    eigenstates = dictionary
    
    Output:
    array of [feasible binary 1D numpyarray,probability of this answer]
    
    """
    bestinarray = sorted(eigenstates.items(), key=lambda item: item[1])[::-1]
    feasible = False
    counter = 0
    total = sum(eigenstates.values())
    final = np.zeros(2)
    
    while counter<len(bestinarray):
        #string to array
        bestasint = np.array([0])
        for i in bestinarray[counter][0]:
            bestasint = np.hstack((bestasint, int(i)))
        feasible = qap_feasible(bestasint[1:])
        if feasible ==True:
            
            frequency = bestinarray[counter][1]
            final = np.vstack((final,[bestasint[1:], frequency/total]))
            feasible = False
        counter += 1
    return final[1:]

This function outputs the VQE solutions

In [8]:
def testing_quantum(machine, ins,SPtrial,circ,SHOT):
    """
    Input:
    machines: the name of available machines
    ins: the QAP instance
    SPtrial: max_trials for SPSA
    circ: 'RA' or 'TL' 
    SHOT: number of shots
    
    Output:
    outmatrix : [time, eigensolution value,machine value,instance]
    """
    
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    print(mdl.export_to_string())
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) 

    #Eigensolver
    ee = NumPyEigensolver(qubitOp_docplex, k=1)
    result1 = ee.run()
    print('Eigen energy:', result1['eigenvalues'])
    print('Eigen QAP objective:', np.real(result1['eigenvalues'])[0] + offset_docplex)

    #VQE
    seed = 10598
    spsa = SPSA(max_trials=SPtrial)
    if circ == 'RA':
        ry = RealAmplitudes(qubitOp_docplex.num_qubits, entanglement='linear')
    if circ == 'TL':
        ry = TwoLocal(qubitOp_docplex.num_qubits, 'ry', 'cz', reps=5, entanglement='linear')
    vqe = VQE(qubitOp_docplex, ry, spsa)

    #from qiskit.providers.ibmq.managed import IBMQJobManager
    #sim_backend = provider.get_backend('ibmq_qasm_simulator')
    # Submit them all to the backend
    #job_manager = IBMQJobManager()
    #job_set_foo = job_manager.run(circs, backend=sim_backend, name='foo')


    backend = Aer.get_backend(machine)
    #backend = provider.get_backend(machine)
    quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed, skip_qobj_validation = False, shots = SHOT)
    result = vqe.run(quantum_instance)

    print('VQE energy:', result['eigenvalue'])
    print('VQE time:', result['optimizer_time'])
    x = choose_best_feasible(result['eigenstate'])
    soln = []
    if x[0]==0:
        print("No feasible solution found")
        return [[],np.array([[[],[],np.inf]])]
    else:
        for i in x:
            soln.append(qap_value(i[0],MatrixFlow,MatrixLoc))
        print('VQE QAP objective:', soln)
        nsoln = len(soln)

        return machine,np.hstack((x,np.array(soln).reshape(nsoln,1))),result['optimizer_time']
    
    

In [9]:
def gridsearch(machine, ins, trials, circuits, shots):
    final = np.array([])
    finalind = np.array([])
    for t in trials:
        for c in circuits:
            for s in shots:
                final = np.append(final,testing_quantum(machine, ins, t, c, s)[1][:,2][0])
                finalind = np.append(finalind,[ t, c, s])
                print(t,c,s)
    
    return finalind[np.argmin(final)]
    
    

In [116]:
machines = 'qasm_simulator'
instances = matrix_size_4

gridsearch(machines, instances, [100,200,300,400], ["RA","TL"], [100,500,1000,1500])

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 + 440 x_2_3*x_3_0 + 220 x_

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
VQE energy: (-123445.49216279994+0j)
VQE time: 34.814326763153076
VQE QAP objective: [824, 858, 920, 838]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
VQE energy: (-145142.23903786638+0j)
VQE time: 41.82404565811157
VQE QAP objective: [930, 1052, 824]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
VQE energy: (-169257.47749069557+0j)
VQE time: 62.954941511154175
VQE QAP objective: [930]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
VQE energy: (-137364.76974609317+0j)
VQE time: 81.6500768661499
No feasible solution found
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
VQE energy: (-155598.9277861303+0j)
VQE time: 110.57983016967773
VQE QAP objective: [1096, 834, 824]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
VQE energy: (-143376.40631888853+0j)
VQE time: 126.32368278503418
VQE QAP objective: [790, 834, 824, 960]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 

'200'

In [11]:
machines = 'qasm_simulator'
instances = matrix_size_3

gridsearch(machines, instances, [100,200,300,400], ["RA","TL"], [100,500,1000,1500])

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End

Eigen energy: [-499640.+0.j]
Eigen QAP objective: 10216.0
VQE energy: (-327770.0647626409+

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


VQE energy: (-319545.5496287335+0j)
VQE time: 4.012700796127319
VQE QAP objective: [17226, 12046, 13346, 17616]
100 RA 500
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_

VQE energy: (-488530.81490976724+0j)
VQE time: 7.450728178024292
VQE QAP objective: [17616, 13346]
200 RA 500
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2

VQE energy: (-404280.9143024558+0j)
VQE time: 11.288507461547852
VQE QAP objective: [10216]
300 RA 500
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_

IndexError: list index out of range

In [12]:
def thirtytrials(machine, ins, t, c, s):
    final = np.zeros(2)
    acount95 = 0
    acount99 = 0
    mcount95 = 0
    mcount99 = 0
    for i in range(30):
        ans = testing_quantum(machine, ins, t, c, s)[1][:,2]
        minimum = np.min(ans)
        final = np.vstack((final,[ans[0],minimum]))
        #SR95
        if ans[0] <= 1.05*790:
            acount95+=1
        if minimum <= 1.05*790:
            mcount95+=1
        #sr99
        if ans[0] <= 1.01*790:
            acount99+=1
        if minimum <= 1.01*790:
            mcount99+=1
    return 100*acount95/30, 100*acount99/30, 100*mcount95/30, 100*mcount99/30

In [14]:
def thirtytrials1(machine, ins, t, c, s):
    final = np.zeros(2)
    acount95 = 0
    acount99 = 0
    mcount95 = 0
    mcount99 = 0
    for i in range(30):
        ans = testing_quantum(machine, ins, t, c, s)[1][:,2]
        minimum = np.min(ans)
        final = np.vstack((final,[ans[0],minimum]))
        #SR95
        if ans[0] <= 1.05*10216:
            acount95+=1
        if minimum <= 1.05*10216:
            mcount95+=1
        #sr99
        if ans[0] <= 1.01*10216:
            acount99+=1
        if minimum <= 1.01*10216:
            mcount99+=1
    return 100*acount95/30, 100*acount99/30, 100*mcount95/30, 100*mcount99/30

In [10]:
t = 100
c = "RA"
s = 1000
machines = 'qasm_simulator'
instances = matrix_size_4

thirtytrials(machines, instances, t, c, s)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 + 440 x_2_3*x_3_0 + 220 x_

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
VQE energy: (-114829.13470605295+0j)
VQE time: 38.252838373184204
VQE QAP objective: [926, 1136, 908]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
VQE energy: (-132229.6126110087+0j)
VQE time: 39.29208040237427
VQE QAP objective: [824, 990]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
VQE energy: (-115692.84165045594+0j)
VQE time: 37.610323905944824
VQE QAP objective: [858]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
VQE energy: (-116766.27519634044+0j)
VQE time: 34.60000801086426
VQE QAP objective: [982]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
VQE energy: (-132016.00598313074+0j)
VQE time: 45.88283348083496
VQE QAP objective: [824, 920]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x

Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
VQE energy: (-127636.79142313155+0j)
VQE time: 41.499231815338135
VQE QAP objective: [920]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1

(16.666666666666668, 0.0, 36.666666666666664, 10.0)

In [15]:
t = 100
c = "RA"
s = 1000
machines = 'qasm_simulator'
instances = matrix_size_3

thirtytrials(machines, instances, t, c, s)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End

Eigen energy: [-499640.+0.j]
Eigen QAP objective: 10216.0
VQE energy: (-379320.0327860046+

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


VQE energy: (-334052.2866037548+0j)
VQE time: 6.940021276473999
VQE QAP objective: [17226, 13346, 12046, 17616]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x

VQE energy: (-355717.7634342949+0j)
VQE time: 6.687167406082153
VQE QAP objective: [13346, 17616]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2

VQE energy: (-444372.1079586102+0j)
VQE time: 4.821230888366699
VQE QAP objective: [13346, 12046]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2

VQE energy: (-397303.28223117715+0j)
VQE time: 4.958175897598267
VQE QAP objective: [17226, 12046]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_

(3.3333333333333335, 3.3333333333333335, 20.0, 20.0)

In [2]:
(38.05276107788086+38.252838373184204+38.70327877998352+36.896085023880005+43.03075408935547+38.42258858680725+39.29208040237427+36.55540943145752+34.541343688964844+34.7874755859375+35.96805739402771+37.610323905944824+39.096577405929565+35.004643201828+35.29738259315491+36.03959035873413+34.60000801086426+37.02075457572937+34.56398844718933+38.124258041381836+43.86992931365967+45.88283348083496+46.29799675941467+43.388824462890625+50.76052951812744+42.026928663253784+41.499231815338135+40.749661922454834+40.803627729415894+40.15799880027771)/30

39.2432587146759

In [16]:
(5.564791440963745+6.940021276473999+6.043537139892578+6.584223985671997+6.984995365142822+6.671175956726074+5.654757261276245+4.891198635101318+5.6427648067474365+6.687167406082153+7.209866046905518+6.0865113735198975+5.996562480926514+4.877224445343018+4.968170642852783+5.018122911453247+4.738282203674316+ 4.821230888366699+4.822234869003296+4.893175840377808+4.97515082359314+4.985143423080444+5.003109455108643+5.068093538284302+4.792253255844116+4.958175897598267+4.868188142776489+4.752275466918945+6.3763649463653564+5.738711833953857)/30

5.553782725334168

In [21]:
t = 100
c = "RA"
s = 1000
machines = 'ibmq_johannesburg'
instances = matrix_size_3

testing_quantum(machines, instances, t, c, s)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 4704 x_0_0*x_1_1 + 3444 x_0_0*x_1_2 + 21280 x_0_0*x_2_1
      + 15580 x_0_0*x_2_2 + 4704 x_0_1*x_1_0 + 504 x_0_1*x_1_2
      + 21280 x_0_1*x_2_0 + 2280 x_0_1*x_2_2 + 3444 x_0_2*x_1_0
      + 504 x_0_2*x_1_1 + 15580 x_0_2*x_2_0 + 2280 x_0_2*x_2_1
      + 18368 x_1_0*x_2_1 + 13448 x_1_0*x_2_2 + 18368 x_1_1*x_2_0
      + 1968 x_1_1*x_2_2 + 13448 x_1_2*x_2_0 + 1968 x_1_2*x_2_1 ]/2
Subject To
 c1: x_0_0 + x_0_1 + x_0_2 = 1
 c2: x_1_0 + x_1_1 + x_1_2 = 1
 c3: x_2_0 + x_2_1 + x_2_2 = 1
 c4: x_0_0 + x_1_0 + x_2_0 = 1
 c5: x_0_1 + x_1_1 + x_2_1 = 1
 c6: x_0_2 + x_1_2 + x_2_2 = 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_0_2 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_1_2 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_2_2 <= 1

Binaries
 x_0_0 x_0_1 x_0_2 x_1_0 x_1_1 x_1_2 x_2_0 x_2_1 x_2_2
End

Eigen energy: [-499640.+0.j]
Eigen QAP objective: 10216.0


FAILURE: job id: 5f0470b07450b4001275875b, status: 'FAIL_TO_GET_STATUS' Terra job error: "'524 Server Error:  for url: https://api-qcon.quantum-computing.ibm.com/api/Network/ibm-q-wits/Groups/internal/Projects/physics/Jobs/5f0470b07450b4001275875b/status'"
FAILURE: job id: 5f0470b07450b4001275875b, status: 'FAIL_TO_GET_STATUS' Terra job error: "'524 Server Error:  for url: https://api-qcon.quantum-computing.ibm.com/api/Network/ibm-q-wits/Groups/internal/Projects/physics/Jobs/5f0470b07450b4001275875b/status'"


VQE energy: (-183808.3420000001+0j)
VQE time: 14238.776804208755
VQE QAP objective: [12046, 17226, 17616, 10216]


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


('ibmq_johannesburg',
 array([[array([0, 1, 0, 0, 0, 1, 1, 0, 0]), 0.026, 12046],
        [array([0, 0, 1, 0, 1, 0, 1, 0, 0]), 0.006, 17226],
        [array([0, 0, 1, 1, 0, 0, 0, 1, 0]), 0.001, 17616],
        [array([0, 1, 0, 1, 0, 0, 0, 0, 1]), 0.001, 10216]], dtype=object),
 14238.776804208755)

### Test Symmetric and Assymmetric cases

In [16]:
machines = 'qasm_simulator'
for n in [3,4]:
    for q in range(5):
        instances = "./data/made_asym" + str(n) + "_" + str(q) + ".csv"
        ans = gridsearch(machines, instances, [100,200,300,400], ["RA","TL"], [100,500,1000,1500])
ans

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 6900 x_0_0^2 + 8970 x_0_0*x_0_1 + 11316 x_0_0*x_0_2 + 4000 x_0_0*x_1_0
      + 3654 x_0_0*x_1_1 + 6068 x_0_0*x_1_2 + 10400 x_0_0*x_2_0
      + 8372 x_0_0*x_2_1 + 12792 x_0_0*x_2_2 + 1380 x_0_1^2 + 17250 x_0_1*x_0_2
      + 1546 x_0_1*x_1_0 + 800 x_0_1*x_1_1 + 4762 x_0_1*x_1_2 + 5148 x_0_1*x_2_0
      + 2080 x_0_1*x_2_1 + 12636 x_0_1*x_2_2 + 4278 x_0_2^2 + 492 x_0_2*x_1_0
      + 5238 x_0_2*x_1_1 + 2480 x_0_2*x_1_2 + 4264 x_0_2*x_2_0
      + 13364 x_0_2*x_2_1 + 6448 x_0_2*x_2_2 + 7900 x_1_0^2 + 10270 x_1_0*x_1_1
      + 12956 x_1_0*x_1_2 + 11500 x_1_0*x_2_0 + 9056 x_1_0*x_2_1
      + 13612 x_1_0*x_2_2 + 1580 x_1_1^2 + 19750 x_1_1*x_1_2 + 5894 x_1_1*x_2_0
      + 2300 x_1_1*x_2_1 + 14018 x_1_1*x_2_2 + 4898 x_1_2^2 + 5248 x_1_2*x_2_0
      + 14732 x_1_2*x_2_1 + 7130 x_1_2*x_2_2 + 600 x_2_0^2 + 780 x_2_0*x_2_1
      + 984 x_2_0*x_2_2 + 120 x_2_1^2 + 1500 x_2_1*x_2_2 + 372 x_2_2^2 ]/2
Subje

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


VQE energy: (-615218.3745088254+0j)
VQE time: 4.366180181503296
VQE QAP objective: [10927, 17605, 20545, 11018]
100 RA 500
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 6900 x_0_0^2 + 8970 x_0_0*x_0_1 + 11316 x_0_0*x_0_2 + 4000 x_0_0*x_1_0
      + 3654 x_0_0*x_1_1 + 6068 x_0_0*x_1_2 + 10400 x_0_0*x_2_0
      + 8372 x_0_0*x_2_1 + 12792 x_0_0*x_2_2 + 1380 x_0_1^2 + 17250 x_0_1*x_0_2
      + 1546 x_0_1*x_1_0 + 800 x_0_1*x_1_1 + 4762 x_0_1*x_1_2 + 5148 x_0_1*x_2_0
      + 2080 x_0_1*x_2_1 + 12636 x_0_1*x_2_2 + 4278 x_0_2^2 + 492 x_0_2*x_1_0
      + 5238 x_0_2*x_1_1 + 2480 x_0_2*x_1_2 + 4264 x_0_2*x_2_0
      + 13364 x_0_2*x_2_1 + 6448 x_0_2*x_2_2 + 7900 x_1_0^2 + 10270 x_1_0*x_1_1
      + 12956 x_1_0*x_1_2 + 11500 x_1_0*x_2_0 + 9056 x_1_0*x_2_1
      + 13612 x_1_0*x_2_2 + 1580 x_1_1^2 + 19750 x_1_1*x_1_2 + 5894 x_1_1*x_2_0
      + 2300 x_1_1*x_2_1 + 14018 x_1_1*x_2_2 + 4898 x_1_2^2 + 5248 x_1_2*x_2_0
      + 14732 x_1_2*x_2_1 + 7130 x

VQE energy: (-707047.3458092564+0j)
VQE time: 6.995624542236328
VQE QAP objective: [19658]
100 TL 1500
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 6900 x_0_0^2 + 8970 x_0_0*x_0_1 + 11316 x_0_0*x_0_2 + 4000 x_0_0*x_1_0
      + 3654 x_0_0*x_1_1 + 6068 x_0_0*x_1_2 + 10400 x_0_0*x_2_0
      + 8372 x_0_0*x_2_1 + 12792 x_0_0*x_2_2 + 1380 x_0_1^2 + 17250 x_0_1*x_0_2
      + 1546 x_0_1*x_1_0 + 800 x_0_1*x_1_1 + 4762 x_0_1*x_1_2 + 5148 x_0_1*x_2_0
      + 2080 x_0_1*x_2_1 + 12636 x_0_1*x_2_2 + 4278 x_0_2^2 + 492 x_0_2*x_1_0
      + 5238 x_0_2*x_1_1 + 2480 x_0_2*x_1_2 + 4264 x_0_2*x_2_0
      + 13364 x_0_2*x_2_1 + 6448 x_0_2*x_2_2 + 7900 x_1_0^2 + 10270 x_1_0*x_1_1
      + 12956 x_1_0*x_1_2 + 11500 x_1_0*x_2_0 + 9056 x_1_0*x_2_1
      + 13612 x_1_0*x_2_2 + 1580 x_1_1^2 + 19750 x_1_1*x_1_2 + 5894 x_1_1*x_2_0
      + 2300 x_1_1*x_2_1 + 14018 x_1_1*x_2_2 + 4898 x_1_2^2 + 5248 x_1_2*x_2_0
      + 14732 x_1_2*x_2_1 + 7130 x_1_2*x_2_2 + 600 x_2

VQE energy: (-895899.4277473238+0j)
VQE time: 10.608287811279297
VQE QAP objective: [18723]
200 TL 500
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 6900 x_0_0^2 + 8970 x_0_0*x_0_1 + 11316 x_0_0*x_0_2 + 4000 x_0_0*x_1_0
      + 3654 x_0_0*x_1_1 + 6068 x_0_0*x_1_2 + 10400 x_0_0*x_2_0
      + 8372 x_0_0*x_2_1 + 12792 x_0_0*x_2_2 + 1380 x_0_1^2 + 17250 x_0_1*x_0_2
      + 1546 x_0_1*x_1_0 + 800 x_0_1*x_1_1 + 4762 x_0_1*x_1_2 + 5148 x_0_1*x_2_0
      + 2080 x_0_1*x_2_1 + 12636 x_0_1*x_2_2 + 4278 x_0_2^2 + 492 x_0_2*x_1_0
      + 5238 x_0_2*x_1_1 + 2480 x_0_2*x_1_2 + 4264 x_0_2*x_2_0
      + 13364 x_0_2*x_2_1 + 6448 x_0_2*x_2_2 + 7900 x_1_0^2 + 10270 x_1_0*x_1_1
      + 12956 x_1_0*x_1_2 + 11500 x_1_0*x_2_0 + 9056 x_1_0*x_2_1
      + 13612 x_1_0*x_2_2 + 1580 x_1_1^2 + 19750 x_1_1*x_1_2 + 5894 x_1_1*x_2_0
      + 2300 x_1_1*x_2_1 + 14018 x_1_1*x_2_2 + 4898 x_1_2^2 + 5248 x_1_2*x_2_0
      + 14732 x_1_2*x_2_1 + 7130 x_1_2*x_2_2 + 600 x_2

...

Eigen energy: [-10946319.5+0.j]
Eigen QAP objective: 36633.0
VQE energy: (-9253541.674821401+0j)
VQE time: 168.8545160293579
VQE QAP objective: [46838]
300 TL 100
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 1512 x_0_0^2 + 10332 x_0_0*x_0_1 + 6720 x_0_0*x_0_2 + 7308 x_0_0*x_0_3
      + 5076 x_0_0*x_1_0 + 17372 x_0_0*x_1_1 + 11326 x_0_0*x_1_2
      + 12336 x_0_0*x_1_3 + 3744 x_0_0*x_2_0 + 10008 x_0_0*x_2_1
      + 3904 x_0_0*x_2_2 + 2424 x_0_0*x_2_3 + 5904 x_0_0*x_3_0
      + 19708 x_0_0*x_3_1 + 12384 x_0_0*x_3_2 + 13164 x_0_0*x_3_3 + 2772 x_0_1^2
      + 6132 x_0_1*x_0_2 + 13524 x_0_1*x_0_3 + 17314 x_0_1*x_1_0
      + 9306 x_0_1*x_1_1 + 10332 x_0_1*x_1_2 + 22672 x_0_1*x_1_3
      + 15576 x_0_1*x_2_0 + 6864 x_0_1*x_2_1 + 3848 x_0_1*x_2_2
      + 19528 x_0_1*x_2_3 + 20636 x_0_1*x_3_0 + 10824 x_0_1*x_3_1
      + 11348 x_0_1*x_3_2 + 26868 x_0_1*x_3_3 + 4116 x_0_2^2 + 9156 x_0_2*x_0_3
      + 11234 x_0_2*x_1_0 + 10254 x_0_2*x_1_1 + 13

Eigen energy: [-10946319.5+0.j]
Eigen QAP objective: 36633.0
VQE energy: (-10540220.71809489+0j)
VQE time: 188.15611052513123
VQE QAP objective: [39824]
300 TL 1500
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 1512 x_0_0^2 + 10332 x_0_0*x_0_1 + 6720 x_0_0*x_0_2 + 7308 x_0_0*x_0_3
      + 5076 x_0_0*x_1_0 + 17372 x_0_0*x_1_1 + 11326 x_0_0*x_1_2
      + 12336 x_0_0*x_1_3 + 3744 x_0_0*x_2_0 + 10008 x_0_0*x_2_1
      + 3904 x_0_0*x_2_2 + 2424 x_0_0*x_2_3 + 5904 x_0_0*x_3_0
      + 19708 x_0_0*x_3_1 + 12384 x_0_0*x_3_2 + 13164 x_0_0*x_3_3 + 2772 x_0_1^2
      + 6132 x_0_1*x_0_2 + 13524 x_0_1*x_0_3 + 17314 x_0_1*x_1_0
      + 9306 x_0_1*x_1_1 + 10332 x_0_1*x_1_2 + 22672 x_0_1*x_1_3
      + 15576 x_0_1*x_2_0 + 6864 x_0_1*x_2_1 + 3848 x_0_1*x_2_2
      + 19528 x_0_1*x_2_3 + 20636 x_0_1*x_3_0 + 10824 x_0_1*x_3_1
      + 11348 x_0_1*x_3_2 + 26868 x_0_1*x_3_3 + 4116 x_0_2^2 + 9156 x_0_2*x_0_3
      + 11234 x_0_2*x_1_0 + 10254 x_0_2*x_1_1 + 

Eigen energy: [-10946319.5+0.j]
Eigen QAP objective: 36633.0
VQE energy: (-8082637.577111858+0j)
VQE time: 141.03587770462036
VQE QAP objective: [39398, 46555, 41170]
400 RA 1000
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 1512 x_0_0^2 + 10332 x_0_0*x_0_1 + 6720 x_0_0*x_0_2 + 7308 x_0_0*x_0_3
      + 5076 x_0_0*x_1_0 + 17372 x_0_0*x_1_1 + 11326 x_0_0*x_1_2
      + 12336 x_0_0*x_1_3 + 3744 x_0_0*x_2_0 + 10008 x_0_0*x_2_1
      + 3904 x_0_0*x_2_2 + 2424 x_0_0*x_2_3 + 5904 x_0_0*x_3_0
      + 19708 x_0_0*x_3_1 + 12384 x_0_0*x_3_2 + 13164 x_0_0*x_3_3 + 2772 x_0_1^2
      + 6132 x_0_1*x_0_2 + 13524 x_0_1*x_0_3 + 17314 x_0_1*x_1_0
      + 9306 x_0_1*x_1_1 + 10332 x_0_1*x_1_2 + 22672 x_0_1*x_1_3
      + 15576 x_0_1*x_2_0 + 6864 x_0_1*x_2_1 + 3848 x_0_1*x_2_2
      + 19528 x_0_1*x_2_3 + 20636 x_0_1*x_3_0 + 10824 x_0_1*x_3_1
      + 11348 x_0_1*x_3_2 + 26868 x_0_1*x_3_3 + 4116 x_0_2^2 + 9156 x_0_2*x_0_3
      + 11234 x_0_2*x_1_0 + 10254 

Eigen energy: [-10946319.5+0.j]
Eigen QAP objective: 36633.0
VQE energy: (-8870751.58316045+0j)
VQE time: 141.8164234161377
VQE QAP objective: [46301, 46475, 46893]
400 TL 500
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 1512 x_0_0^2 + 10332 x_0_0*x_0_1 + 6720 x_0_0*x_0_2 + 7308 x_0_0*x_0_3
      + 5076 x_0_0*x_1_0 + 17372 x_0_0*x_1_1 + 11326 x_0_0*x_1_2
      + 12336 x_0_0*x_1_3 + 3744 x_0_0*x_2_0 + 10008 x_0_0*x_2_1
      + 3904 x_0_0*x_2_2 + 2424 x_0_0*x_2_3 + 5904 x_0_0*x_3_0
      + 19708 x_0_0*x_3_1 + 12384 x_0_0*x_3_2 + 13164 x_0_0*x_3_3 + 2772 x_0_1^2
      + 6132 x_0_1*x_0_2 + 13524 x_0_1*x_0_3 + 17314 x_0_1*x_1_0
      + 9306 x_0_1*x_1_1 + 10332 x_0_1*x_1_2 + 22672 x_0_1*x_1_3
      + 15576 x_0_1*x_2_0 + 6864 x_0_1*x_2_1 + 3848 x_0_1*x_2_2
      + 19528 x_0_1*x_2_3 + 20636 x_0_1*x_3_0 + 10824 x_0_1*x_3_1
      + 11348 x_0_1*x_3_2 + 26868 x_0_1*x_3_3 + 4116 x_0_2^2 + 9156 x_0_2*x_0_3
      + 11234 x_0_2*x_1_0 + 10254 x_0

'100'

In [ ]:
#3_0: 100 RA 100, 10927
#3_1: 100 RA 1000, 17713
#3_2: 100 RA 100, 15148
#3_3: 100 TL 500, 22882
#3_4: 100 TL 500, 20966
#4_0: 100 TL 100, 26246
#4_1: 200 TL 1500, 33643
#4_2: 200 RA 100, 37341
#4_3: 200 RA 1000, 42918
#4_4: 400 RA 1500, 36633

In [10]:
def thirtytrials2(known,machine, ins, t, c, s):
    final = np.zeros(2)
    acount95 = 0
    acount99 = 0
    mcount95 = 0
    mcount99 = 0
    for i in range(30):
        ans = testing_quantum(machine, ins, t, c, s)[1][:,2]
        minimum = np.min(ans)
        final = np.vstack((final,[ans[0],minimum]))
        #SR95
        if ans[0] <= 1.05*known:
            acount95+=1
        if minimum <= 1.05*known:
            mcount95+=1
        #sr99
        if ans[0] <= 1.01*known:
            acount99+=1
        if minimum <= 1.01*known:
            mcount99+=1
    return 100*acount95/30, 100*acount99/30, 100*mcount95/30, 100*mcount99/30

In [11]:
optimal_parameters = np.array([[[100, "RA", 100],[100, "RA", 1000],[100, "RA", 100],[100, "TL", 500],[100, "TL", 500]],
                               [[100, "TL", 100],[200, "TL", 1500],[200, "RA", 100],[200, "RA", 1000],[400, "RA", 1500]]])
knowns = np.array([[10927, 17713, 15148, 22882, 20966], [26246, 33643, 37341, 42918, 36633]])
machine = 'qasm_simulator'
ans = []
for n in [3,4]:
    for q in range(5):
        instances = "./data/made_asym" + str(n) + "_" + str(q) + ".csv"
        ans.append(thirtytrials2(knowns[n-3][q],machine, instances, int(optimal_parameters[n-3][q][0]), optimal_parameters[n-3][q][1], int(optimal_parameters[n-3][q][2])))
ans

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 6900 x_0_0^2 + 8970 x_0_0*x_0_1 + 11316 x_0_0*x_0_2 + 4000 x_0_0*x_1_0
      + 3654 x_0_0*x_1_1 + 6068 x_0_0*x_1_2 + 10400 x_0_0*x_2_0
      + 8372 x_0_0*x_2_1 + 12792 x_0_0*x_2_2 + 1380 x_0_1^2 + 17250 x_0_1*x_0_2
      + 1546 x_0_1*x_1_0 + 800 x_0_1*x_1_1 + 4762 x_0_1*x_1_2 + 5148 x_0_1*x_2_0
      + 2080 x_0_1*x_2_1 + 12636 x_0_1*x_2_2 + 4278 x_0_2^2 + 492 x_0_2*x_1_0
      + 5238 x_0_2*x_1_1 + 2480 x_0_2*x_1_2 + 4264 x_0_2*x_2_0
      + 13364 x_0_2*x_2_1 + 6448 x_0_2*x_2_2 + 7900 x_1_0^2 + 10270 x_1_0*x_1_1
      + 12956 x_1_0*x_1_2 + 11500 x_1_0*x_2_0 + 9056 x_1_0*x_2_1
      + 13612 x_1_0*x_2_2 + 1580 x_1_1^2 + 19750 x_1_1*x_1_2 + 5894 x_1_1*x_2_0
      + 2300 x_1_1*x_2_1 + 14018 x_1_1*x_2_2 + 4898 x_1_2^2 + 5248 x_1_2*x_2_0
      + 14732 x_1_2*x_2_1 + 7130 x_1_2*x_2_2 + 600 x_2_0^2 + 780 x_2_0*x_2_1
      + 984 x_2_0*x_2_2 + 120 x_2_1^2 + 1500 x_2_1*x_2_2 + 372 x_2_2^2 ]/2
Subje

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


VQE energy: (-639358.1189806389+0j)
VQE time: 3.9957144260406494
VQE QAP objective: [11018, 20545, 10927]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 6900 x_0_0^2 + 8970 x_0_0*x_0_1 + 11316 x_0_0*x_0_2 + 4000 x_0_0*x_1_0
      + 3654 x_0_0*x_1_1 + 6068 x_0_0*x_1_2 + 10400 x_0_0*x_2_0
      + 8372 x_0_0*x_2_1 + 12792 x_0_0*x_2_2 + 1380 x_0_1^2 + 17250 x_0_1*x_0_2
      + 1546 x_0_1*x_1_0 + 800 x_0_1*x_1_1 + 4762 x_0_1*x_1_2 + 5148 x_0_1*x_2_0
      + 2080 x_0_1*x_2_1 + 12636 x_0_1*x_2_2 + 4278 x_0_2^2 + 492 x_0_2*x_1_0
      + 5238 x_0_2*x_1_1 + 2480 x_0_2*x_1_2 + 4264 x_0_2*x_2_0
      + 13364 x_0_2*x_2_1 + 6448 x_0_2*x_2_2 + 7900 x_1_0^2 + 10270 x_1_0*x_1_1
      + 12956 x_1_0*x_1_2 + 11500 x_1_0*x_2_0 + 9056 x_1_0*x_2_1
      + 13612 x_1_0*x_2_2 + 1580 x_1_1^2 + 19750 x_1_1*x_1_2 + 5894 x_1_1*x_2_0
      + 2300 x_1_1*x_2_1 + 14018 x_1_1*x_2_2 + 4898 x_1_2^2 + 5248 x_1_2*x_2_0
      + 14732 x_1_2*x_2_1 + 7130 x_1_2*x_2_2 + 600 

VQE energy: (-571847.7355253148+0j)
VQE time: 3.6999664306640625
VQE QAP objective: [10927]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 6900 x_0_0^2 + 8970 x_0_0*x_0_1 + 11316 x_0_0*x_0_2 + 4000 x_0_0*x_1_0
      + 3654 x_0_0*x_1_1 + 6068 x_0_0*x_1_2 + 10400 x_0_0*x_2_0
      + 8372 x_0_0*x_2_1 + 12792 x_0_0*x_2_2 + 1380 x_0_1^2 + 17250 x_0_1*x_0_2
      + 1546 x_0_1*x_1_0 + 800 x_0_1*x_1_1 + 4762 x_0_1*x_1_2 + 5148 x_0_1*x_2_0
      + 2080 x_0_1*x_2_1 + 12636 x_0_1*x_2_2 + 4278 x_0_2^2 + 492 x_0_2*x_1_0
      + 5238 x_0_2*x_1_1 + 2480 x_0_2*x_1_2 + 4264 x_0_2*x_2_0
      + 13364 x_0_2*x_2_1 + 6448 x_0_2*x_2_2 + 7900 x_1_0^2 + 10270 x_1_0*x_1_1
      + 12956 x_1_0*x_1_2 + 11500 x_1_0*x_2_0 + 9056 x_1_0*x_2_1
      + 13612 x_1_0*x_2_2 + 1580 x_1_1^2 + 19750 x_1_1*x_1_2 + 5894 x_1_1*x_2_0
      + 2300 x_1_1*x_2_1 + 14018 x_1_1*x_2_2 + 4898 x_1_2^2 + 5248 x_1_2*x_2_0
      + 14732 x_1_2*x_2_1 + 7130 x_1_2*x_2_2 + 600 x_2_0^2 + 780 

VQE energy: (-573613.2508553367+0j)
VQE time: 3.795550584793091
No feasible solution found
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap3

Minimize
 obj: [ 6900 x_0_0^2 + 8970 x_0_0*x_0_1 + 11316 x_0_0*x_0_2 + 4000 x_0_0*x_1_0
      + 3654 x_0_0*x_1_1 + 6068 x_0_0*x_1_2 + 10400 x_0_0*x_2_0
      + 8372 x_0_0*x_2_1 + 12792 x_0_0*x_2_2 + 1380 x_0_1^2 + 17250 x_0_1*x_0_2
      + 1546 x_0_1*x_1_0 + 800 x_0_1*x_1_1 + 4762 x_0_1*x_1_2 + 5148 x_0_1*x_2_0
      + 2080 x_0_1*x_2_1 + 12636 x_0_1*x_2_2 + 4278 x_0_2^2 + 492 x_0_2*x_1_0
      + 5238 x_0_2*x_1_1 + 2480 x_0_2*x_1_2 + 4264 x_0_2*x_2_0
      + 13364 x_0_2*x_2_1 + 6448 x_0_2*x_2_2 + 7900 x_1_0^2 + 10270 x_1_0*x_1_1
      + 12956 x_1_0*x_1_2 + 11500 x_1_0*x_2_0 + 9056 x_1_0*x_2_1
      + 13612 x_1_0*x_2_2 + 1580 x_1_1^2 + 19750 x_1_1*x_1_2 + 5894 x_1_1*x_2_0
      + 2300 x_1_1*x_2_1 + 14018 x_1_1*x_2_2 + 4898 x_1_2^2 + 5248 x_1_2*x_2_0
      + 14732 x_1_2*x_2_1 + 7130 x_1_2*x_2_2 + 600 x_2_0^2 + 780 x

...

Eigen energy: [-10946319.5+0.j]
Eigen QAP objective: 36633.0
VQE energy: (-7725302.420855875+0j)
VQE time: 204.77389287948608
VQE QAP objective: [48894, 48550]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 1512 x_0_0^2 + 10332 x_0_0*x_0_1 + 6720 x_0_0*x_0_2 + 7308 x_0_0*x_0_3
      + 5076 x_0_0*x_1_0 + 17372 x_0_0*x_1_1 + 11326 x_0_0*x_1_2
      + 12336 x_0_0*x_1_3 + 3744 x_0_0*x_2_0 + 10008 x_0_0*x_2_1
      + 3904 x_0_0*x_2_2 + 2424 x_0_0*x_2_3 + 5904 x_0_0*x_3_0
      + 19708 x_0_0*x_3_1 + 12384 x_0_0*x_3_2 + 13164 x_0_0*x_3_3 + 2772 x_0_1^2
      + 6132 x_0_1*x_0_2 + 13524 x_0_1*x_0_3 + 17314 x_0_1*x_1_0
      + 9306 x_0_1*x_1_1 + 10332 x_0_1*x_1_2 + 22672 x_0_1*x_1_3
      + 15576 x_0_1*x_2_0 + 6864 x_0_1*x_2_1 + 3848 x_0_1*x_2_2
      + 19528 x_0_1*x_2_3 + 20636 x_0_1*x_3_0 + 10824 x_0_1*x_3_1
      + 11348 x_0_1*x_3_2 + 26868 x_0_1*x_3_3 + 4116 x_0_2^2 + 9156 x_0_2*x_0_3
      + 11234 x_0_2*x_1_0 + 10254 x_0_2*x_1_1 + 13818

Eigen energy: [-10946319.5+0.j]
Eigen QAP objective: 36633.0
VQE energy: (-9115781.055725835+0j)
VQE time: 149.33485317230225
VQE QAP objective: [43326, 46301]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 1512 x_0_0^2 + 10332 x_0_0*x_0_1 + 6720 x_0_0*x_0_2 + 7308 x_0_0*x_0_3
      + 5076 x_0_0*x_1_0 + 17372 x_0_0*x_1_1 + 11326 x_0_0*x_1_2
      + 12336 x_0_0*x_1_3 + 3744 x_0_0*x_2_0 + 10008 x_0_0*x_2_1
      + 3904 x_0_0*x_2_2 + 2424 x_0_0*x_2_3 + 5904 x_0_0*x_3_0
      + 19708 x_0_0*x_3_1 + 12384 x_0_0*x_3_2 + 13164 x_0_0*x_3_3 + 2772 x_0_1^2
      + 6132 x_0_1*x_0_2 + 13524 x_0_1*x_0_3 + 17314 x_0_1*x_1_0
      + 9306 x_0_1*x_1_1 + 10332 x_0_1*x_1_2 + 22672 x_0_1*x_1_3
      + 15576 x_0_1*x_2_0 + 6864 x_0_1*x_2_1 + 3848 x_0_1*x_2_2
      + 19528 x_0_1*x_2_3 + 20636 x_0_1*x_3_0 + 10824 x_0_1*x_3_1
      + 11348 x_0_1*x_3_2 + 26868 x_0_1*x_3_3 + 4116 x_0_2^2 + 9156 x_0_2*x_0_3
      + 11234 x_0_2*x_1_0 + 10254 x_0_2*x_1_1 + 13818

Eigen energy: [-10946319.5+0.j]
Eigen QAP objective: 36633.0
VQE energy: (-9161332.634048423+0j)
VQE time: 149.99614357948303
VQE QAP objective: [43326, 48550, 48894]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 1512 x_0_0^2 + 10332 x_0_0*x_0_1 + 6720 x_0_0*x_0_2 + 7308 x_0_0*x_0_3
      + 5076 x_0_0*x_1_0 + 17372 x_0_0*x_1_1 + 11326 x_0_0*x_1_2
      + 12336 x_0_0*x_1_3 + 3744 x_0_0*x_2_0 + 10008 x_0_0*x_2_1
      + 3904 x_0_0*x_2_2 + 2424 x_0_0*x_2_3 + 5904 x_0_0*x_3_0
      + 19708 x_0_0*x_3_1 + 12384 x_0_0*x_3_2 + 13164 x_0_0*x_3_3 + 2772 x_0_1^2
      + 6132 x_0_1*x_0_2 + 13524 x_0_1*x_0_3 + 17314 x_0_1*x_1_0
      + 9306 x_0_1*x_1_1 + 10332 x_0_1*x_1_2 + 22672 x_0_1*x_1_3
      + 15576 x_0_1*x_2_0 + 6864 x_0_1*x_2_1 + 3848 x_0_1*x_2_2
      + 19528 x_0_1*x_2_3 + 20636 x_0_1*x_3_0 + 10824 x_0_1*x_3_1
      + 11348 x_0_1*x_3_2 + 26868 x_0_1*x_3_3 + 4116 x_0_2^2 + 9156 x_0_2*x_0_3
      + 11234 x_0_2*x_1_0 + 10254 x_0_2*x_1_1 

Eigen energy: [-10946319.5+0.j]
Eigen QAP objective: 36633.0
VQE energy: (-8965376.189035045+0j)
VQE time: 161.94523286819458
VQE QAP objective: [48894, 41170]
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 1512 x_0_0^2 + 10332 x_0_0*x_0_1 + 6720 x_0_0*x_0_2 + 7308 x_0_0*x_0_3
      + 5076 x_0_0*x_1_0 + 17372 x_0_0*x_1_1 + 11326 x_0_0*x_1_2
      + 12336 x_0_0*x_1_3 + 3744 x_0_0*x_2_0 + 10008 x_0_0*x_2_1
      + 3904 x_0_0*x_2_2 + 2424 x_0_0*x_2_3 + 5904 x_0_0*x_3_0
      + 19708 x_0_0*x_3_1 + 12384 x_0_0*x_3_2 + 13164 x_0_0*x_3_3 + 2772 x_0_1^2
      + 6132 x_0_1*x_0_2 + 13524 x_0_1*x_0_3 + 17314 x_0_1*x_1_0
      + 9306 x_0_1*x_1_1 + 10332 x_0_1*x_1_2 + 22672 x_0_1*x_1_3
      + 15576 x_0_1*x_2_0 + 6864 x_0_1*x_2_1 + 3848 x_0_1*x_2_2
      + 19528 x_0_1*x_2_3 + 20636 x_0_1*x_3_0 + 10824 x_0_1*x_3_1
      + 11348 x_0_1*x_3_2 + 26868 x_0_1*x_3_3 + 4116 x_0_2^2 + 9156 x_0_2*x_0_3
      + 11234 x_0_2*x_1_0 + 10254 x_0_2*x_1_1 + 13818

[(36.666666666666664,
  36.666666666666664,
  53.333333333333336,
  53.333333333333336),
 (30.0, 30.0, 50.0, 50.0),
 (26.666666666666668, 16.666666666666668, 40.0, 30.0),
 (16.666666666666668,
  6.666666666666667,
  26.666666666666668,
  16.666666666666668),
 (73.33333333333333, 20.0, 90.0, 36.666666666666664),
 (13.333333333333334, 0.0, 16.666666666666668, 3.3333333333333335),
 (6.666666666666667, 6.666666666666667, 10.0, 10.0),
 (13.333333333333334, 10.0, 13.333333333333334, 10.0),
 (10.0, 3.3333333333333335, 13.333333333333334, 6.666666666666667),
 (6.666666666666667, 6.666666666666667, 20.0, 20.0)]

In [21]:
optimal_parameters = np.array([[[100, "RA", 100],[100, "RA", 1000],[100, "RA", 100],[100, "TL", 500],[100, "TL", 500]],[
                               [100, "TL", 100],[200, "TL", 1500],[200, "RA", 100],[200, "RA", 1000],[400, "RA", 1500]]])

In [ ]:
machines = 'qasm_simulator'
for n in [3,4]:
    for q in range(5):
        instances = "./data/made_sym" + str(n) + "_" + str(q) + ".csv"
        ans = gridsearch(machines, instances, [100,200,300,400], ["RA","TL"], [100,500,1000,1500])
ans